<br>
<br>

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Modelos seq2seq 2**

Cuando la entrada de cada paso del decodificador proviene de la salida del paso anterior, estamos hablando de un modelo Seq2Seq con realimentación (feedback). Esto es especialmente común en tareas como la generación de texto.

1. **Inicio de la Secuencia**: Se inicia la generación con un token especial, como `<SOS>` (Start of Sequence).

2. **Generación Paso a Paso**:
   - En el primer paso, el decodificador recibe el `<SOS>` y el estado del codificador como entrada.
   - El decodificador procesa esta entrada y genera una salida para este paso.
   - La salida generada se convierte en la entrada para el siguiente paso, junto con el estado actualizado del decodificador.
   - Este proceso se repite hasta que se genera un token especial de fin de secuencia (`<EOS>`, End of Sequence) o hasta alcanzar un límite máximo de longitud.

3. **Ventajas y Desventajas**:
   - **Ventajas**: Esta forma de generar secuencias puede ayudar a mantener la coherencia en las secuencias generadas, ya que cada nueva palabra o token tiene en cuenta lo que ya se ha generado.
   - **Desventajas**: Puede ser más lento, ya que cada paso depende del anterior, y errores en un paso pueden propagarse y afectar los pasos siguientes.


Imagina que tienes un modelo seq2seq entrenado para traducir inglés a español. Para la frase "How are you?", el proceso sería algo así:

1. El codificador procesa "How are you?" y genera un vector latente.
2. El decodificador recibe el vector latente y el token `<SOS>`.
3. El decodificador genera "¿Cómo", actualiza su estado y toma "¿Cómo" como entrada para el siguiente paso.
4. El decodificador genera "estás", actualiza su estado y toma "estás" como entrada para el siguiente paso.
5. Y así sucesivamente, hasta generar `<EOS>` para indicar el final de la secuencia.

Este modelo de generación permite que el decodificador tenga en cuenta no solo el contexto proporcionado por el codificador, sino también la estructura de la secuencia que está generando, paso a paso.


<p align="center">
<img src="imgs/seq2seq_feedback.svg" width="70%">
</p>


### **Teacher Forcing**

**Teacher forcing** es una técnica utilizada en el entrenamiento de modelos seq2seq en la que, en un porcentaje de las veces, se utiliza la salida real (etiqueta) de un paso de tiempo como entrada para el siguiente paso, en lugar de la salida predicha por el modelo. Esta técnica puede ayudar a acelerar la convergencia y mejorar el rendimiento del modelo, especialmente en las etapas iniciales del entrenamiento.

#### **¿Cómo funciona?**

1. **Durante el Entrenamiento**: En cada paso de tiempo y con una cierta probabilidad de que suceda, en lugar de pasar la predicción del modelo del paso anterior como entrada al siguiente paso, se pasa la palabra real de la secuencia objetivo. Esto proporciona al modelo información directa y clara sobre cómo debería haber respondido en el paso anterior, independientemente de si la predicción fue correcta o no.

2. **Durante la Evaluación/Predicción**: El modelo debe generar secuencias por sí mismo, utilizando sus propias predicciones del paso anterior para el siguiente paso. Durante esta fase, no se utiliza "teacher forcing".

#### **Ventajas de Teacher Forcing:**

1. **Aprendizaje más Rápido**: Al proporcionar al modelo la respuesta correcta en cada paso, se reduce la propagación de errores a través de la secuencia, lo que puede llevar a un aprendizaje más rápido.

2. **Mejor Rendimiento**: Puede resultar en un mejor rendimiento del modelo, especialmente en las primeras etapas del entrenamiento.

### **Implementación traductor inglés a español**

#### **Dataset Europarl**

El conjunto de datos Europarl contiene las transcripciones de los procedimientos del Parlamento Europeo, proporcionando una valiosa fuente de textos paralelos en 21 idiomas europeos. Las oraciones están alineadas entre los idiomas, lo que lo hace especialmente útil para tareas de traducción automática. 

Descargamos el dataset Europarl para español-inglés. Una vez descargado tendremos dos ficheros de texto, uno para cada idioma con las frases alineadas. El código siguiente muestra las primeras frases de cada fichero.

In [1]:
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import torchtext
import torch
from collections import defaultdict


In [2]:
class Translation(Dataset):
    def __init__(self, source_file, target_file):
        self.ingles = []
        self.espanol = []
        self.tokenizer_es = get_tokenizer("spacy", language="es_core_news_md")
        self.tokenizer_en = get_tokenizer("spacy", language="en_core_web_md")
        self.vocab_es = torchtext.vocab.FastText(language='es', unk_init=torch.Tensor.normal_)  # <-- Mirar esto para ver si añadir el token <unk> al vocabulario
        self.vocab_en = torchtext.vocab.FastText(language='en', unk_init=torch.Tensor.normal_)

        self.vocab_en = self.add_sos_eos_unk_pad(self.vocab_en)
        self.vocab_es = self.add_sos_eos_unk_pad(self.vocab_es)

        self.archivo_ingles = source_file
        self.archivo_espanol = target_file

        # Leer el conjunto de datos
        for ingles, espanol in self.read_translation():
            self.ingles.append(ingles)
            self.espanol.append(espanol)


    def add_sos_eos_unk_pad(self, vocabulary):
        words = vocabulary.itos
        vocab = vocabulary.stoi
        embedding_matrix = vocabulary.vectors

        # Tokens especiales
        sos_token = '<sos>'
        eos_token = '<eos>'
        pad_token = '<pad>'
        unk_token = '<unk>'

        # Inicializamos los vectores para los tokens especiales, por ejemplo, con ceros
        sos_vector = torch.full((1, embedding_matrix.shape[1]), 1.)
        eos_vector = torch.full((1, embedding_matrix.shape[1]), 2.)
        pad_vector = torch.zeros((1, embedding_matrix.shape[1]))
        unk_vector = torch.full((1, embedding_matrix.shape[1]), 3.)

        # Añade los vectores al final de la matriz de embeddings
        embedding_matrix = torch.cat((embedding_matrix, sos_vector, eos_vector, unk_vector, pad_vector), 0)

        # Añade los tokens especiales al vocabulario
        vocab[sos_token] = len(vocab)
        vocab[eos_token] = len(vocab)
        vocab[pad_token] = len(vocab)
        vocab[unk_token] = len(vocab)

        words.append(sos_token)
        words.append(eos_token)
        words.append(pad_token)
        words.append(unk_token)

        vocabulary.itos = words
        vocabulary.stoi = vocab
        vocabulary.vectors = embedding_matrix

        default_stoi = defaultdict(lambda : len(vocabulary)-1, vocabulary.stoi)
        vocabulary.stoi = default_stoi
    
        return vocabulary
        

    def read_translation(self):
        with open(self.archivo_ingles, 'r', encoding='utf-8') as f_ingles, open(self.archivo_espanol, 'r', encoding='utf-8') as f_espanol:
            for oracion_ingles, oracion_espanol in zip(f_ingles, f_espanol):
                yield oracion_ingles.strip().lower(), oracion_espanol.strip().lower()

    def __len__(self):
        return len(self.ingles)

    def __getitem__(self, idx):
        item = self.ingles[idx], self.espanol[idx]
        tokens_ingles = self.tokenizer_en(item[0])
        tokens_espanol = self.tokenizer_es(item[1])

        tokens_ingles = tokens_ingles + ['<eos>']
        tokens_espanol = ['<sos>'] + tokens_espanol + ['<eos>']

        if not tokens_ingles or not tokens_espanol:
            return torch.zeros(1, 300), torch.zeros(1, 300)
            # raise RuntimeError("Una de las muestras está vacía.")
    
        tensor_ingles = self.vocab_en.get_vecs_by_tokens(tokens_ingles)
        tensor_espanol = self.vocab_es.get_vecs_by_tokens(tokens_espanol)

        indices_ingles = [self.vocab_en.stoi[token] for token in tokens_ingles] + [self.vocab_en.stoi['<pad>']]
        indices_espanol = [self.vocab_es.stoi[token] for token in tokens_espanol] + [self.vocab_es.stoi['<pad>']]

        return tensor_ingles, tensor_espanol, indices_ingles, indices_espanol
        
            
        
def collate_fn(batch):
    ingles_batch, espanol_batch, ingles_seqs, espanol_seqs = zip(*batch)
    ingles_batch = pad_sequence(ingles_batch, batch_first=True, padding_value=0)
    espanol_batch = pad_sequence(espanol_batch, batch_first=True, padding_value=0)

    # Calcular la longitud máxima de la lista de listas de índices
    pad = espanol_seqs[0][-1]  # token <pad>
    max_len = max([len(l) for l in espanol_seqs])
    for seq in espanol_seqs:
        seq += [pad]*(max_len-len(seq))
        
    return ingles_batch, espanol_batch, ingles_seqs, espanol_seqs

In [3]:

archivo_ingles = 'mock.en'
archivo_espanol = 'mock.es'

translation = Translation(archivo_ingles, archivo_espanol)

#### **Modelo**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
# Parámetros
input_dim = 300
output_dim = 985671
hidden_dim = 512
num_layers = 1
learning_rate = 0.001
num_epochs = 30
batch_size = 8
num_workers = 0
shuffle = True

##### **Encoder**

In [5]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout=0.1):
        super().__init__() 
        
        self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        
    def forward(self, x):
        print("\nEncoder")
        output, (hidden, cell) = self.rnn(x)
        print("Encoder output shape: ", output.shape)
    
        return output, (hidden, cell)
    
encoder = Encoder(input_dim, hidden_dim, num_layers)

##### **Decoder**

In [6]:
class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True)  # TO DO: Añadir dropout 
        self.fc_out = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x, hidden, cell):
        print("\nDecoder")
        output, (hidden, cell) = self.rnn(x, (hidden, cell))
        output = self.fc_out(output)
        return output, (hidden, cell)

In [7]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(BahdanauAttention, self).__init__()

    def forward(self, query, value):
        print("\nAttention")
        attn_energy = torch.sum(query*value, dim=2)

        return F.softmax(attn_energy, dim=1).unsqueeze(1)

In [8]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.es_embeddings = nn.Embedding.from_pretrained(translation.vocab_es.vectors, freeze=True)
        self.M = torch.cat((self.es_embeddings.weight, torch.zeros((0, self.es_embeddings.weight.shape[1]))), 0)
        self.attention = BahdanauAttention(hidden_dim)
        self.concat = nn.Linear(512*2, 512)
        self.out = nn.Linear(512, 985671)

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        print("\n\nSeq2Seq")
        print("Source shape: ", source.shape) # (batch_size, seq_len, input_dim)
        print("Target shape: ", target.shape) # (batch_size, seq_len, input_dim)
        print("\nVamos a entrar en el encoder")
        encoder_outputs, (hidden, cell) = self.encoder(source)
        print("\nHemos salido del encoder")
        batch_size = target.shape[0]
        target_len = target.shape[1]
        outputs = []
        contexts = []


        x = target[:, 0, :]
        print("X shape: ", x.shape)
        for t in range(0, target_len):
            print("\nVamos a entrar en el decoder")
            print("X.unsqueeze(1):",x.unsqueeze(1).shape, "\nHidden", hidden.shape, "\nCell:",cell.shape)
            
            output, (decoder_hidden, cell) = self.decoder(x.unsqueeze(1), hidden, cell)
            
            print("\nHemos salido del decoder")
            print("Output shape: ", output.shape)
            print("Hidden shape: ", hidden.shape)
            print("Cell shape: ", cell.shape)
            print("\nVamos a entrar en la atención")
            
            attention_weights = self.attention(output, encoder_outputs)
            context = attention_weights.bmm(encoder_outputs)

            print("\nHemos salido de la atención")

            teacher_force = 0 < teacher_forcing_ratio
            if teacher_force:
                x = target[:, t, :]
            else:

                x = torch.matmul(context.squeeze(1), self.M)
            outputs.append(output)
            contexts.append(context)
        print("Outputs shape: ", outputs)
        outputs = torch.cat(outputs, dim=1)
        contexts = torch.cat(contexts, dim=1)
        print("Outputs shape: ", outputs.shape)
        print("Contexts shape: ", contexts.shape)    

        concat_input =  torch.cat((outputs, contexts), 2)
        concat_output = torch.tanh(self.concat(concat_input))
            # return output predict
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        return output, hidden


In [9]:
decoder = Decoder(input_dim, hidden_dim, output_dim)


In [10]:
model = Seq2Seq(encoder, decoder)


In [11]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# DataLoader
from torch.utils.data import DataLoader

dataloader = DataLoader(translation, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate_fn)

c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Bucle de entrenamiento
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (src, tgt, src_indices, tgt_indices) in enumerate(dataloader):
        optimizer.zero_grad()
        output,hidden = model(src, tgt)
        print("\n\nSalimos de Seq2Seq")
        print("Output shape: ", output.shape)
        tgt_indices = torch.tensor(tgt_indices, dtype=torch.long)
        loss = 0
        for t in range(1, tgt.shape[1]):
            print(tgt_indices)
            if t == 1:
                exit()
            loss += criterion(output[:, t, :], tgt_indices[:, t])
        # loss = criterion(output, torch.tensor(tgt_indices, dtype=torch.long))

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if batch_idx % 5 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / len(dataloader):.4f}')




Seq2Seq
Source shape:  torch.Size([7, 3, 300])
Target shape:  torch.Size([7, 4, 300])

Vamos a entrar en el encoder

Encoder
Encoder output shape:  torch.Size([7, 3, 512])

Hemos salido del encoder
X shape:  torch.Size([7, 300])

Vamos a entrar en el decoder
X.unsqueeze(1): torch.Size([7, 1, 300]) 
Hidden torch.Size([1, 7, 512]) 
Cell: torch.Size([1, 7, 512])

Decoder

Hemos salido del decoder
Output shape:  torch.Size([7, 1, 512])
Hidden shape:  torch.Size([1, 7, 512])
Cell shape:  torch.Size([1, 7, 512])

Vamos a entrar en la atención

Attention

Hemos salido de la atención

Vamos a entrar en el decoder
X.unsqueeze(1): torch.Size([7, 1, 300]) 
Hidden torch.Size([1, 7, 512]) 
Cell: torch.Size([1, 7, 512])

Decoder

Hemos salido del decoder
Output shape:  torch.Size([7, 1, 512])
Hidden shape:  torch.Size([1, 7, 512])
Cell shape:  torch.Size([1, 7, 512])

Vamos a entrar en la atención

Attention

Hemos salido de la atención

Vamos a entrar en el decoder
X.unsqueeze(1): torch.Size([7, 

KeyboardInterrupt: 

: 